In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Concatenate, BatchNormalization, Activation
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
import tensorflow as tf
import keras
from sklearn.model_selection import KFold, StratifiedKFold

Using TensorFlow backend.


In [3]:
def auc(y_true, y_pred):
    f1 = lambda: tf.constant(0, dtype=tf.float64)
    f2 = lambda: tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    
    r = tf.case([(tf.equal(tf.reduce_sum(y_true), tf.constant(0, dtype=tf.float32)), f1),
                 (tf.equal(tf.reduce_sum(tf.subtract(tf.ones_like(y_true), y_true)), tf.constant(0, dtype=tf.float32)), f1)
                ], default=f2)
    return r

In [4]:
df_train = pd.read_csv('train_with_counts.csv')
y = df_train['target'].values
df_train_X = df_train.drop(columns=['ID_code', 'target'])
df_train_X_normalized = (df_train_X - df_train_X.mean(axis=0))/df_train_X.std(axis=0)

In [5]:
df_train_X_normalized.head()

,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,...,var_190_FE,var_191_FE,var_192_FE,var_193_FE,var_194_FE,var_195_FE,var_196_FE,var_197_FE,var_198_FE,var_199_FE
0,-0.577100,-1.273734,0.460090,1.750966,-0.020872,0.491725,-0.692319,1.624021,2.104313,0.488123,...,0.006398,2.010107,0.654904,0.514849,2.809453,-0.579089,1.740723,0.871209,0.520762,0.042849
1,0.269958,-0.622136,-0.144986,-0.772678,-1.011350,1.527378,0.761581,1.865887,-1.895580,-0.673628,...,3.051745,0.542409,0.654904,-0.679281,0.162290,-0.883793,1.017512,2.429459,1.001115,0.042849
2,-0.681112,-0.276066,-0.130752,0.675978,-0.157928,-0.858653,-0.989044,1.785603,-0.286601,-1.714792,...,0.615468,0.542409,-0.241114,-0.679281,-0.896576,-1.493201,0.294300,0.425994,0.040410,0.042849
3,0.125158,-0.129425,0.347543,-1.444036,1.841835,-0.426742,-1.173890,-1.106680,0.045292,0.935693,...,-0.602671,-0.436055,-0.241114,0.514849,0.162290,0.335022,1.740723,-1.354863,-0.920295,0.042849
4,-0.277303,0.035610,-1.788859,-1.778093,-0.213578,-0.574155,0.678888,-2.228820,1.108634,0.108366,...,0.006398,2.010107,-1.734476,-1.276346,-1.426008,0.639725,1.740723,-0.241827,-0.920295,0.042849


In [6]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df_train_X, df_train_y, batch_size=256, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.df_train_X = df_train_X
        self.df_train_y = df_train_y
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        return self.X_train[indexes], self.y_train[indexes]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        print('epoch ended')
        self.X_train, self.y_train = shuffle_data(self.df_train_X, self.df_train_y)
#         self.X_train = self.df_train_X.values
#         self.y_train = self.df_train_y
        self.indexes = np.arange(len(self.X_train))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [7]:
def shuffle_data(x, y):
    x1 = x[y==1]
    X_ones = np.zeros(x1.shape)
    ids = np.arange(x1.shape[0])
    for c in range(200):
        np.random.shuffle(ids)
        X_ones[:, [c,c+200]] = x1[[f'var_{c}', f'var_{c}_FE']].values[ids]
    
    x0 = x[y==0].copy() 
    X_zeros = np.zeros(x0.shape)
    ids = np.arange(x0.shape[0])
    for c in range(200):
        np.random.shuffle(ids)
        X_zeros[:, [c,c+200]] = x0[[f'var_{c}', f'var_{c}_FE']].values[ids]

    return np.append(X_zeros, X_ones, axis=0), np.append(np.zeros(len(X_zeros)), np.ones(len(X_ones)))

In [8]:
def get_model():
    model = Sequential()
    model.add(Dense(1000, input_shape=(400,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [9]:
model = get_model()

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              401000    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 402,001
Trainable params: 402,001
Non-trainable params: 0
_________________________________________________________________


In [11]:
best_model_file_name = 'best_full_model.h5'

In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
patience = 1000
epochs = 1000
bs = 512
for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train_X_normalized, y)):
    print('###############################################')
    print(f'##################Fold {fold}#######################')
    print('###############################################')
    X_val = df_train_X_normalized.values[val_idx]
    
    print('Got train and val data!')
    lrs = [0.001, 0.0001, 0.00005]
    model = get_model()
    generator = DataGenerator(df_train_X_normalized.iloc[trn_idx], y[trn_idx], batch_size=bs)
    for lr in lrs:
        model.compile(Adam(lr=lr, decay=0), loss='binary_crossentropy', metrics=[auc])
        es = EarlyStopping(monitor='val_auc', patience=patience, mode='max', verbose=1)
        mc = ModelCheckpoint(best_model_file_name, monitor='val_auc', mode='max', verbose=1, save_best_only=True)
        # First train
        model.fit_generator(generator, 
              epochs=epochs,
              verbose=1,
              callbacks = [es, 
                           mc], 
                           #clr],
              validation_data=(X_val, y[val_idx]))
        print(f'Finish training with lr {lr}')
        model = get_model()
        # Load weights from ModelCheckpoint
        model.load_weights(best_model_file_name)
        # Save them to disk
        model.save_weights(f'big_NN_simple_fold_{fold}_lr_{lr}.hdf5')
        print('weights saved')

###############################################
##################Fold 0#######################
###############################################
Got train and val data!
epoch ended
Epoch 1/1000
4999/4999 [==============================] - 10s 2ms/step - loss: 0.2509 - auc: 0.8119 - val_loss: 0.4491 - val_auc: 0.4963

Epoch 00001: val_auc improved from -inf to 0.49633, saving model to best_full_model.h5
Epoch 2/1000
4999/4999 [==============================] - 10s 2ms/step - loss: 0.2437 - auc: 0.8191 - val_loss: 0.4501 - val_auc: 0.4972

Epoch 00002: val_auc improved from 0.49633 to 0.49722, saving model to best_full_model.h5
Epoch 3/1000
2975/4999 [================>.............] - ETA: 3s - loss: 0.2423 - auc: 0.8285

KeyboardInterrupt: 